In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

!pip install lightgbm

In [2]:
test_set = pd.read_csv('test_engineered_features.csv')
train_set = pd.read_csv('train_engineered_features.csv')
labelsDF = pd.read_csv('train_labels.csv')
train_set

,Unnamed: 0,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,...,damage_grade,more_families_than_mean,older_than_mean,cant_materials,mean_damage_per_count_floor,mean_damage_level_1,height per area,mean_damage_level_1_per_plan_configuration,most_damaged_region,less_damaged_region
0,0,802906,6,487,12198,2,30,6,5,t,...,3,False,True,2,2.24,2.161724,0.833333,2.169963,0,0
1,1,633759,6,706,12267,2,5,7,5,t,...,2,False,False,1,2.24,2.161724,0.714286,2.169963,0,0
2,2,157370,6,706,11722,2,20,4,4,t,...,2,False,False,1,2.24,2.161724,1.000000,2.169963,0,0
3,3,33651,6,1210,9423,3,40,5,7,t,...,3,False,True,1,2.40,2.161724,1.400000,2.169963,0,0
4,4,1005736,6,724,7981,2,10,5,5,t,...,2,False,False,1,2.24,2.161724,1.000000,2.169963,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,260596,665334,29,903,10668,1,20,9,3,n,...,2,False,False,1,2.04,2.078283,0.333333,1.800000,0,0
260597,260597,506593,29,903,10668,3,55,7,7,n,...,2,False,True,2,2.40,2.078283,1.000000,1.800000,0,0
260598,260598,994870,29,903,10668,1,30,10,4,n,...,2,False,True,1,2.04,2.078283,0.400000,1.800000,0,0
260599,260599,345899,29,903,10668,2,25,15,6,t,...,1,False,False,1,2.24,2.078283,0.400000,1.800000,0,0


In [3]:
test_set["land_surface_condition"]               = test_set["land_surface_condition"].astype("category")
test_set["foundation_type"]                      = test_set["foundation_type"].astype("category")
test_set["roof_type"]                            = test_set["roof_type"].astype("category")
test_set["ground_floor_type"]                    = test_set["ground_floor_type"].astype("category")
test_set["other_floor_type"]                     = test_set["other_floor_type"].astype("category")
test_set["position"]                             = test_set["position"].astype("category")
test_set["plan_configuration"]                   = test_set["plan_configuration"].astype("category")
test_set["legal_ownership_status"]               = test_set["legal_ownership_status"].astype("category")

train_set["land_surface_condition"]               = train_set["land_surface_condition"].astype("category")
train_set["foundation_type"]                      = train_set["foundation_type"].astype("category")
train_set["roof_type"]                            = train_set["roof_type"].astype("category")
train_set["ground_floor_type"]                    = train_set["ground_floor_type"].astype("category")
train_set["other_floor_type"]                     = train_set["other_floor_type"].astype("category")
train_set["position"]                             = train_set["position"].astype("category")
train_set["plan_configuration"]                   = train_set["plan_configuration"].astype("category")
train_set["legal_ownership_status"]               = train_set["legal_ownership_status"].astype("category")

In [4]:
train_set = train_set.loc[:, train_set.columns != 'Unnamed: 0']
train_set.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,damage_grade,more_families_than_mean,older_than_mean,cant_materials,mean_damage_per_count_floor,mean_damage_level_1,height per area,mean_damage_level_1_per_plan_configuration,most_damaged_region,less_damaged_region
0,802906,6,487,12198,2,30,6,5,t,r,...,3,False,True,2,2.24,2.161724,0.833333,2.169963,0,0
1,633759,6,706,12267,2,5,7,5,t,r,...,2,False,False,1,2.24,2.161724,0.714286,2.169963,0,0
2,157370,6,706,11722,2,20,4,4,t,r,...,2,False,False,1,2.24,2.161724,1.000000,2.169963,0,0
3,33651,6,1210,9423,3,40,5,7,t,r,...,3,False,True,1,2.40,2.161724,1.400000,2.169963,0,0
4,1005736,6,724,7981,2,10,5,5,t,r,...,2,False,False,1,2.24,2.161724,1.000000,2.169963,0,0


In [5]:
train_set.columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_i

In [6]:
X1, y1 =  train_set[['geo_level_1_id', 
                     #'geo_level_2_id', 
                     #'geo_level_3_id',
                    'count_floors_pre_eq', 
                     #'age', 
                     'area_percentage', 
                     'height_percentage',
                   'land_surface_condition', 
                     'foundation_type', 
                     'roof_type',
       'ground_floor_type', 
        #             'other_floor_type', 
         #            'position',
       'plan_configuration', 
        #             'has_superstructure_adobe_mud',
       #'has_superstructure_mud_mortar_stone', 
        #             'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       #'has_superstructure_mud_mortar_brick',
       #'has_superstructure_cement_mortar_brick', 
       #              'has_superstructure_timber',
       #'has_superstructure_bamboo', 
       #              'has_superstructure_rc_non_engineered',
       #'has_superstructure_rc_engineered', 
       #              'has_superstructure_other',
       #'legal_ownership_status', 
       #              'count_families', 
       #              'has_secondary_use',
       #'has_secondary_use_agriculture',
       #              'has_secondary_use_hotel',
       #'has_secondary_use_rental', 
       #              'has_secondary_use_institution',
       #'has_secondary_use_school', 
       #              'has_secondary_use_industry',
       #'has_secondary_use_health_post', 
       #              'has_secondary_use_gov_office',
       #'has_secondary_use_use_police', 
       #              'has_secondary_use_other',
       # 'more_families_than_mean', 
       #              'older_than_mean',
       'cant_materials', 
                     'mean_damage_per_count_floor', 
                     'mean_damage_level_1',
       'height per area'
                    ]], train_set['damage_grade']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

In [8]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = [0,1],)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [9]:
lgb_clas = lgb.LGBMClassifier(boosting_type = 'gbdt', 
                                   verbose_eval = 20,
                              early_stopping_rounds = 3,
                              n_jobs=4)

In [10]:
lgb_clas.fit(X_train,y_train,eval_set=(X_train, y_train))

C:\Users\alasi\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] early_stopping_round is set=3, early_stopping_rounds=3 will be ignored. Current value: early_stopping_round=3
[1]	training's multi_logloss: 0.871119
Training until validation scores don't improve for 3 rounds
[2]	training's multi_logloss: 0.841093
[3]	training's multi_logloss: 0.817658
[4]	training's multi_logloss: 0.798962
[5]	training's multi_logloss: 0.783727
[6]	training's multi_logloss: 0.771132
[7]	training's multi_logloss: 0.760556
[8]	training's multi_logloss: 0.751716
[9]	training's multi_logloss: 0.744206
[10]	training's multi_logloss: 0.737812
[11]	training's multi_logloss: 0.732371
[12]	training's multi_logloss: 0.727624
[13]	training's multi_logloss: 0.723537
[14]	training's multi_logloss: 0.71995
[15]	training's multi_logloss: 0.716761
[16]	training's multi_logloss: 0.713912
[17]	training's multi_logloss: 0.711399
[18]	training's multi_logloss: 0.709203
[19]	training's multi_logloss: 0.707192
[20]	t

LGBMClassifier(early_stopping_rounds=3, n_jobs=4, verbose_eval=20)

In [11]:
predicciones_train = lgb_clas.predict(X_test)
predicciones_train

array([3, 2, 2, ..., 3, 1, 2], dtype=int64)

In [12]:
from sklearn.metrics import f1_score
f1_score(y_test, predicciones_train, average='micro')

0.6768652230081478

In [13]:
# Usando el set de test otorgado
predicciones_test = lgb_clas.predict(test_set[['geo_level_1_id', 
                     #'geo_level_2_id', 
                     #'geo_level_3_id',
                    'count_floors_pre_eq', 
                     #'age', 
                     'area_percentage', 
                     'height_percentage',
                   'land_surface_condition', 
                     'foundation_type', 
                    # 'roof_type',
       #'ground_floor_type', 
        #             'other_floor_type', 
         #            'position',
       #'plan_configuration', 
        #             'has_superstructure_adobe_mud',
       #'has_superstructure_mud_mortar_stone', 
        #             'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       #'has_superstructure_mud_mortar_brick',
       #'has_superstructure_cement_mortar_brick', 
       #              'has_superstructure_timber',
       #'has_superstructure_bamboo', 
       #              'has_superstructure_rc_non_engineered',
       #'has_superstructure_rc_engineered', 
       #              'has_superstructure_other',
       #'legal_ownership_status', 
       #              'count_families', 
       #              'has_secondary_use',
       #'has_secondary_use_agriculture',
       #              'has_secondary_use_hotel',
       #'has_secondary_use_rental', 
       #              'has_secondary_use_institution',
       #'has_secondary_use_school', 
       #              'has_secondary_use_industry',
       #'has_secondary_use_health_post', 
       #              'has_secondary_use_gov_office',
       #'has_secondary_use_use_police', 
       #              'has_secondary_use_other',
       # 'more_families_than_mean', 
       #              'older_than_mean',
       'cant_materials', 
                     'mean_damage_per_count_floor', 
                     'mean_damage_level_1',
       'height per area'
                    ]])

ValueError: Number of features of the model must match the input. Model n_features_ is 14 and input n_features is 11 

In [ ]:
predicciones_test

In [ ]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predicciones_test,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('PrediccionLightGBM.csv')

# Resultado 1:
- Modelo : LiGBM
- hiperparams: default,no hubo busqueda de hiper parametros

- Features utilizados:
        
        'building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other',
        'more_families_than_mean', 'older_than_mean',
       'cant_materials', 'mean_damage_per_count_floor', 'mean_damage_level_1',
       'height per area'


- Score : 0.4748